In [14]:
import numpy as np
import pandas as pd
import os, tempfile
os.chdir(tempfile.mkdtemp())
pd.DataFrame(np.random.random((100,10)),columns=[f'c{i}' for i in range(10)]).to_csv('data.csv',index=False)
!head -2 data.csv

c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0.4886828340952287,0.23340297158812018,0.8306029298126891,0.16930188648456113,0.6356797581133748,0.07497655044464024,0.03429700767456523,0.22667428354653196,0.4147789663205891,0.2857651211208935


# sql
* https://plot.ly/ipython-notebooks/big-data-analytics-with-pandas-and-sqlite/

# dask
* https://jakevdp.github.io/blog/2015/08/14/out-of-core-dataframes-in-python/

In [16]:
from dask import dataframe as dd
data = dd.read_csv('data.csv', usecols=['c1','c5'])
data

,c1,c5
npartitions=1,,
,float64,float64
,...,...


In [20]:
print(data.c5.count())
print(dd.compute(data.c5.count()))

dd.Scalar<series-..., dtype=int64>
(100,)


# datatable

In [ ]:
import datatable as dt
print(dt.__version__)

In [ ]:
df = dt.fread("data.csv")
df.to_numpy()
df.to_pandas()

In [ ]:
df.shape
df.names # list of column names
df.stypes # list of column types
df.head(10)

In [ ]:
df.sum()
df.sd()
df.mode()
df.nmodal()
df.nunique()
df.max()
df.min()
df.mean()

In [ ]:
df[:,'column_name']
df[:5,:3]
df.sort('column_name')
df[dt.f.loan_amnt>dt.f.funded_amnt,"loan_amnt"]

In [ ]:
del df[:,'column_name']

In [ ]:
# groupby
for i in range(100):
    df[:, dt.sum(dt.f.funded_amnt), dt.by(dt.f.grade)]
    
## equiv pandas command
for i in range(100):
    pandas_df.groupby("grade")["funded_amnt"].sum()

In [ ]:
df.to_csv('filename.csv')